In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1013. Microbiome Peanut Allergy/P1013.3. Analyses/P1013.4. ML Prediction/Output')
cur_date = "112025"

library(readxl)
library(openxlsx)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)
library(themis)

# reading in files
saliva_df = data.frame(read_excel("Input/Processed_Saliva_Data_090525.xlsx")) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE)) 
stool_df = data.frame(read_excel("Input/Processed_Stool_Data_090525.xlsx")) %>%
    mutate(Age = as.numeric(Age), IgE = as.numeric(IgE))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift



Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching packa

In [2]:
head(saliva_df)

,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,k__Bacteria.__.__.__.__.__.__,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS01baseline,1,Pre_1,Pre,P,F,3.838356,41.00,High,7,⋯,57,13,0,213,775,0,14,246,44,64
2,FS0236mo,2,Post_2,Post,A,DS,1.873973,14.20,Low,63,⋯,698,0,43,98,1657,0,0,423,146,123
3,FS02baseline,2,Pre_2,Pre,A,DS,1.873973,14.20,Low,0,⋯,0,0,46,371,3359,0,0,0,0,112
4,FS0436mo,3,Post_3,Post,P,F,3.715068,67.40,High,34,⋯,0,0,0,0,92,0,200,67,32,201
5,FS04baseline,3,Pre_3,Pre,P,F,3.715068,67.40,High,4,⋯,0,0,0,0,43,414,47,0,2,1050
6,FS0536mo,4,Post_4,Post,A,DS,3.991781,9.59,Low,6,⋯,0,17,0,139,2521,0,0,502,39,101


Using supervised ML to see if the outcome can be predicted using global microbiome abundances.

1. Predicting outcome using baseline abundances with participants given active treatment
2. Predicting IgE
3. Predicting outcome with paired samples given active treatment

In [3]:
combined_saliva_df = saliva_df %>%
    # normalizing
    mutate(across(10:ncol(saliva_df), ~ log(.x + 2))) %>%
    # removing those failed the IgE protein
    filter(IgE_Binarized != 'Fail') %>%
    # ended up reassinging 'DS' to 'F' due to small sample size
    mutate(Outcome = relevel(factor(ifelse(Outcome %in% 'SU', 1, 0)), ref = "0"),
           IgE_Binarized = relevel(factor(ifelse(IgE_Binarized == 'Low', 0, 1)), ref = "0")) 

combined_stool_df = stool_df  %>%
    mutate(across(11:ncol(stool_df), ~ log(.x + 2))) %>%
    filter(IgE_Binarized != 'Fail') %>%
    mutate(Outcome = relevel(factor(ifelse(Outcome %in% 'SU', 1, 0)), ref = "0"),
           IgE_Binarized = relevel(factor(ifelse(IgE_Binarized == 'Low', 0, 1)), ref = "0")) 

head(combined_stool_df)

,Original_Subject_ID,Original_Sample_ID,Subject_No,Sample_ID,Time_Point,Treatment,Outcome,Age,IgE,IgE_Binarized,⋯,d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Lactobacillales.f__Streptococcaceae.g__Streptococcus.__,d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Staphylococcales.f__Gemellaceae.g__Gemella.__,d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Staphylococcales.f__Staphylococcaceae.g__Staphylococcus.__,d__Bacteria.p__Patescibacteria.__.__.__.__.__,d__Bacteria.p__Patescibacteria.c__Saccharimonadia.o__Saccharimonadales.f__Nanogingivalaceae.g__Nanogingivalis.s__Nanogingivalis.gingivitcus,d__Bacteria.p__Patescibacteria.c__Saccharimonadia.o__Saccharimonadales.f__Nanosynbacteraceae.g__Nanosynbacter.s__,d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Burkholderiales_595427.f__Burkholderiaceae_A_595427.g__Parasutterella.__,d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales_A_737866.f__Enterobacteriaceae_A.__.__,d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales_A_737866.f__Enterobacteriaceae_A.g__Escherichia_710834.__,d__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales_A_737866.f__Pasteurellaceae.g__Haemophilus_D_735815.__
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,FS001-02,FS001-02-36mo,1,Post_1,Post,A,0,1.873973,14.20,0,⋯,7.344073,3.4657359,3.4657359,0.6931472,0.6931472,1.9459101,0.6931472,0.6931472,5.153292,0.6931472
2,FS001-02,FS001-02-BL,1,Pre_1,Pre,A,0,1.873973,14.20,0,⋯,6.969791,1.7917595,0.6931472,0.6931472,1.6094379,0.6931472,0.6931472,3.1780538,2.484907,0.6931472
3,FS001-04,FS001-04-36mo,2,Post_2,Post,P,0,3.715068,67.40,1,⋯,9.022202,1.9459101,5.8171112,0.6931472,1.6094379,0.6931472,0.6931472,0.6931472,7.793174,0.6931472
4,FS001-04,FS001-04-BL,2,Pre_2,Pre,P,0,3.715068,67.40,1,⋯,7.789040,1.6094379,3.2958369,0.6931472,2.1972246,0.6931472,0.6931472,2.4849066,8.843182,0.6931472
5,FS001-05,FS001-05-36mo,3,Post_3,Post,A,0,3.991781,9.59,0,⋯,6.326149,0.6931472,2.1972246,0.6931472,1.3862944,3.9702919,4.1431347,0.6931472,8.086103,1.6094379
6,FS001-06,FS001-06-36mo,4,Post_4,Post,P,0,2.356164,106.00,1,⋯,4.882802,0.6931472,0.6931472,0.6931472,0.6931472,3.7612001,0.6931472,0.6931472,4.043051,0.6931472


# Variable Selection

To determine which variables are collinear and need to be removed, but first doing this with continuous data only to run Spearman's correlation tests. 

In [4]:
# creating correlation matrix
corr_matrix_saliva = rcorr(as.matrix(combined_saliva_df[,10:ncol(combined_saliva_df)]), type = "spearman")
corr_matrix_stool = rcorr(as.matrix(combined_stool_df[,10:ncol(combined_stool_df)]), type = "spearman")

# creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_saliva_df = data.frame(corr_matrix_saliva$r) %>%
  rownames_to_column(var = "Variable1") %>%
  # needed to specify package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  # also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  # removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

corr_matrix_stool_df = data.frame(corr_matrix_stool$r) %>%
  rownames_to_column(var = "Variable1") %>%
  # needed to specify package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  # also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  # removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

corr_matrix_saliva_df$Variable2 = as.character(corr_matrix_saliva_df$Variable2)
corr_matrix_stool_df$Variable2 = as.character(corr_matrix_stool_df$Variable2)

Using Variable1 as id variables

Using Variable1 as id variables



In [5]:
# counting number of occurences
occurences_saliva_df = full_join(data.frame(table(corr_matrix_saliva_df$Variable1)), 
                           data.frame(table(corr_matrix_saliva_df$Variable2))) %>%
    arrange(-Freq) %>%
    # removing variables with greater than 5 occurences
    filter(Freq >= 5)

occurences_stool_df = full_join(data.frame(table(corr_matrix_stool_df$Variable1)), 
                           data.frame(table(corr_matrix_stool_df$Variable2))) %>%
    arrange(-Freq) %>%
    # removing variables with greater than 2 occurences
    filter(Freq >= 2)

occurences_saliva_df
occurences_stool_df

Joining with `by = join_by(Var1, Freq)`
Joining with `by = join_by(Var1, Freq)`


Var1,Freq
<fct>,<int>
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Peptostreptococcaceae.g__Mogibacterium.__,10
k__Bacteria.p__Actinobacteria.c__Coriobacteriia.o__Coriobacteriales.f__Atopobiaceae.g__Lancefieldella.__,10
k__Bacteria.p__Firmicutes.c__Negativicutes.o__Veillonellales.f__Veillonellaceae.g__Megasphaera.s__micronuciformis,8
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Prevotella.s__pallens,8
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Peptostreptococcaceae.g__Peptostreptococcaceae_.G.1..s__.Eubacterium._sulci,7
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Prevotella.__,6
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Prevotella.s__melaninogenica,6
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Prevotella.s__pallens,5
k__Bacteria.p__Firmicutes.c__Erysipelotrichia.o__Erysipelotrichales.f__Erysipelotrichaceae.g__Solobacterium.s__moorei,5


Var1,Freq
<fct>,<int>


In [6]:
# getting different dataframes for each model
baseline_saliva_df = combined_saliva_df %>%
    filter(Time_Point == 'Pre' & Treatment == 'A') #%>%
    #select(!occurences_saliva_df$Var1)

baseline_stool_df = combined_stool_df #%>%
    #filter(Time_Point == 'Pre' & Treatment == 'A') 

v1_saliva_df = baseline_saliva_df[,c(6,10:ncol(baseline_saliva_df))]
v1_stool_df = baseline_stool_df[,c(7,11:ncol(baseline_stool_df))]
v1_saliva_cov_df = baseline_saliva_df[,c(6:8,10:ncol(baseline_saliva_df))]
v1_stool_cov_df = baseline_stool_df[,c(7:9,11:ncol(baseline_stool_df))]
v2_saliva_df = combined_saliva_df[,9:ncol(combined_saliva_df)] %>%
    select(!occurences_saliva_df$Var1)
v2_stool_df = combined_stool_df[,10:ncol(combined_stool_df)] 
v2_saliva_cov_df = combined_saliva_df[,c(7,9:ncol(combined_saliva_df))] %>%
    select(!occurences_saliva_df$Var1)
v2_stool_cov_df = combined_stool_df[,c(8,10:ncol(combined_stool_df))] 

head(v1_saliva_cov_df)

,Outcome,Age,IgE,k__Bacteria.__.__.__.__.__.__,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_874,k__Bacteria.p__Absconditabacteria_.SR1..c__Absconditabacteria_.SR1._.C.1..o__Absconditabacteria_.SR1._.O.1..f__Absconditabacteria_.SR1._.F.1..g__Absconditabacteria_.SR1._.G.1..s__bacterium_HMT_875,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Actinomyces.__,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Actinomyces.s__graevenitzii,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Schaalia.s__lingnae,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Schaalia.s__sp._HMT_180,⋯,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.__.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Aggregatibacter.s__aphrophilus,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.s__parainfluenzae,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..__,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_347,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.1..s__bacterium_HMT_352,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.3..s__bacterium_HMT_351,k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1..g__Saccharibacteria_.TM7._.G.6..s__bacterium_HMT_870
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1.873973,14.20,0.6931472,2.8903718,0.6931472,3.663562,4.2766661,4.0073332,5.049856,⋯,0.6931472,0.6931472,3.8712010,5.921578,8.119994,0.6931472,0.6931472,0.6931472,0.6931472,4.736198
2,0,3.991781,9.59,3.2580965,0.6931472,2.3025851,5.327876,3.3322045,4.0430513,6.173786,⋯,0.6931472,0.6931472,0.6931472,4.718499,7.112327,3.7376696,1.7917595,6.0038871,3.6109179,5.129899
3,1,1.419178,1.32,2.1972246,0.6931472,0.6931472,3.988984,2.3025851,0.6931472,3.806662,⋯,0.6931472,0.6931472,0.6931472,7.748029,7.590852,0.6931472,2.8903718,0.6931472,1.9459101,3.401197
4,1,1.389041,0.42,2.3978953,0.6931472,0.6931472,3.828641,0.6931472,0.6931472,4.795791,⋯,0.6931472,0.6931472,0.6931472,7.280697,8.360773,0.6931472,0.6931472,0.6931472,0.6931472,2.197225
5,0,1.567123,4.34,2.3025851,0.6931472,0.6931472,3.135494,2.5649494,4.7273878,5.541264,⋯,0.6931472,0.6931472,0.6931472,2.890372,8.445482,0.6931472,0.6931472,3.6635616,2.1972246,3.496508
6,0,3.912329,105.00,1.3862944,0.6931472,0.6931472,3.988984,5.7235851,4.8362819,7.167038,⋯,2.3025851,0.6931472,0.6931472,3.555348,5.575949,0.6931472,0.6931472,7.2909748,4.0943446,4.060443


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [7]:
rf_classification = function(dataset, outcome, pred_outcome, knn){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    ntree_values = c(50, 250, 500) # number of trees 
    p = dim(dataset)[2] - 1 # number of variables in dataset
    mtry_values = c(sqrt(p), p/2, p/3) # number of predictors

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome, k = knn)
        data_test = dataset[dataset_index[[i]],]

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors = which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        #extracting accuracy, sens, spec, PPV, NPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:4])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), NPV = mean(Neg.Pred.Value), 
              AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini) #%>%
        # cleaning up names
        # separate(Predictor, c("Name", "Genus_Species"), sep = '.g__') %>%
        # mutate(across(Genus_Species, ~ str_replace(string = .x, ".s__", " ")),
        #       across(Genus_Species, ~ str_replace(string = .x, "sp._", "")),
        #       across(Genus_Species, ~ str_replace(string = .x, ".__", "")),
        #       Genus_Species = ifelse(Name == 'k__Bacteria.__.__.__.__.__.__' , 'Unassigned', Genus_Species))
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(balanced_data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [8]:
# calling fn
rf_values_bl_saliva = rf_classification(v1_saliva_df, "Outcome", "pred_Outcome", 3)
rf_values_bl_stool = rf_classification(v1_stool_df, "Outcome", "pred_Outcome", 3)
rf_values_bl_saliva_covariates = rf_classification(v1_saliva_cov_df, "Outcome", "pred_Outcome", 3)
rf_values_bl_stool_covariates = rf_classification(v1_stool_cov_df, "Outcome", "pred_Outcome", 3)
rf_values_ige_saliva = rf_classification(v2_saliva_df, "IgE_Binarized", "pred_IgE_Binarized", 5)
rf_values_ige_stool = rf_classification(v2_stool_df, "IgE_Binarized", "pred_IgE_Binarized", 5)
rf_values_ige_saliva_covariates = rf_classification(v2_saliva_cov_df, "IgE_Binarized", "pred_IgE_Binarized", 5)
rf_values_ige_stool_covariates = rf_classification(v2_stool_cov_df, "IgE_Binarized", "pred_IgE_Binarized", 5)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls > cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [9]:
# viewing results
rf_confusion_matrix_bl = data.frame(Model = c(rep(c("RF Classification"), times = 4)), 
                                 Response = c(rep(c("Outcome"), times = 4)), 
                                 Sample = c(rep(c("Saliva"), times = 2), rep(c("Stool"), times = 2)),
                                 Covariates = c(rep(c("No", "Yes"), times = 2)), Kernel = NA, rbind(rf_values_bl_saliva[[2]], 
                                rf_values_bl_saliva_covariates[[2]], rf_values_bl_stool[[2]], rf_values_bl_stool_covariates[[2]]))
rf_confusion_matrix_bl

# viewing most significant features
head(rf_values_bl_saliva[[3]])
head(rf_values_bl_saliva_covariates[[3]])
head(rf_values_bl_stool[[3]]) 
head(rf_values_bl_stool_covariates[[3]])

Model,Response,Sample,Covariates,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,NPV,AUC
<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,Outcome,Saliva,No,NA,0.4500000,0.4000000,0.5000000,NaN,NaN,0.6500000
RF Classification,Outcome,Saliva,Yes,NA,0.6500000,0.7000000,0.6000000,0.7666667,NaN,0.6500000
RF Classification,Outcome,Stool,No,NA,0.5375000,0.1666667,0.9083333,0.4000000,0.7206061,0.5375000
RF Classification,Outcome,Stool,Yes,NA,0.6777778,0.4000000,0.9555556,NaN,0.8141414,0.6777778


Predictor,MeanDecreaseGini
<chr>,<dbl>
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Lachnospiraceae.g__Stomatobaculum.s__sp._HMT_097,0.4540641
k__Bacteria.p__Firmicutes.c__Bacilli.o__Lactobacillales.f__Streptococcaceae.g__Streptococcus.__,0.2921732
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.__,0.2774242
k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Pasteurellales.f__Pasteurellaceae.g__Haemophilus.__,0.2701720
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.s__pasteri,0.2522313
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Lachnospiraceae.g__Lachnoanaerobaculum.s__umeaense,0.2409639


Predictor,MeanDecreaseGini
<chr>,<dbl>
IgE,0.9803697
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Lachnospiraceae.g__Lachnoanaerobaculum.s__umeaense,0.2905105
k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Schaalia.s__lingnae,0.2894477
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Alloprevotella.s__sp._HMT_308,0.2421086
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae.g__Porphyromonas.__,0.2300403
k__Bacteria.p__Firmicutes.c__Clostridia.o__Eubacteriales.f__Lachnospiraceae.g__Stomatobaculum.s__sp._HMT_097,0.2296599


Predictor,MeanDecreaseGini
<chr>,<dbl>
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Eubacteriales_258471.f__Eubacteriaceae_258449.g__Eubacterium_O_258270.__,2.7789949
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Oscillospirales.f__Ruminococcaceae.g__Ruminococcus_D.s__Ruminococcus_D.bicirculans,1.0380609
d__Bacteria.p__Actinobacteriota.c__Coriobacteriia.o__Coriobacteriales.f__Coriobacteriaceae.g__Collinsella.__,0.8705610
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Peptostreptococcales.f__Peptostreptococcaceae_256921.g__Intestinibacter.__,0.7031403
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Oscillospirales.f__Ruminococcaceae.g__Ruminiclostridium_E.s__Ruminiclostridium_E.siraeum,0.6518453
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Mediterraneibacter_A_155507.s__Mediterraneibacter_A_155507.faecis,0.6516400


Predictor,MeanDecreaseGini
<chr>,<dbl>
IgE,5.4462127
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Eubacteriales_258471.f__Eubacteriaceae_258449.g__Eubacterium_O_258270.__,1.7086586
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae.g__Mediterraneibacter_A_155507.s__Mediterraneibacter_A_155507.faecis,0.7816464
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Peptostreptococcales.f__Peptostreptococcaceae_256921.g__Intestinibacter.__,0.7673286
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Oscillospirales.f__Ruminococcaceae.g__Ruminococcus_D.s__Ruminococcus_D.bicirculans,0.6873147
d__Bacteria.p__Actinobacteriota.c__Coriobacteriia.o__Coriobacteriales.f__Coriobacteriaceae.g__Collinsella.__,0.5650860


In [18]:
# viewing results
rf_confusion_matrix_ige = data.frame(Model = c(rep(c("RF Classification"), times = 4)), 
                                 Response = c(rep(c("IgE_Binarized"), times = 4)), 
                                 Sample = c(rep(c("Saliva"), times = 2), rep(c("Stool"), times = 2)),
                                 Covariates = c(rep(c("No", "Yes"), times = 2)), Kernel = NA, rbind(rf_values_ige_saliva[[2]], 
                                rf_values_ige_saliva_covariates[[2]], rf_values_ige_stool[[2]], rf_values_ige_stool_covariates[[2]]))
rf_confusion_matrix_ige

# viewing most significant features
head(rf_values_ige_saliva[[3]])
head(rf_values_ige_saliva_covariates[[3]])
head(rf_values_ige_stool[[3]]) 
head(rf_values_ige_stool_covariates[[3]])

Model,Response,Sample,Covariates,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,NPV,AUC
<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,IgE_Binarized,Saliva,No,NA,0.6300000,0.41,0.8500000,0.6333333,0.7103030,0.6300000
RF Classification,IgE_Binarized,Saliva,Yes,NA,0.6175000,0.41,0.8250000,0.6166667,0.7019697,0.6175000
RF Classification,IgE_Binarized,Stool,No,NA,0.5425000,0.36,0.7250000,0.4466667,0.6453968,0.5425000
RF Classification,IgE_Binarized,Stool,Yes,NA,0.5435714,0.28,0.8071429,NaN,0.6338095,0.5435714


Name,Genus_Species,MeanDecreaseGini
<chr>,<chr>,<dbl>
k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales.__.__.__,NA,1.3998322
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae,Porphyromonas HMT_930,1.1758835
k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Corynebacteriales.f__Corynebacteriaceae,Corynebacterium amycolatum,1.1656814
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae,Alloprevotella HMT_473,1.0051959
k__Bacteria.p__Saccharibacteria_.TM7..c__Saccharibacteria_.TM7._.C.1..o__Saccharibacteria_.TM7._.O.1..f__Saccharibacteria_.TM7._.F.1.,Saccharibacteria_.TM7._.G.6. bacterium_HMT_870,0.6441038
k__Bacteria.p__Firmicutes.c__Negativicutes.o__Veillonellales.f__Veillonellaceae,Veillonella parvula,0.6401783


Name,Genus_Species,MeanDecreaseGini
<chr>,<chr>,<dbl>
k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales.__.__.__,NA,1.7946571
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Porphyromonadaceae,Porphyromonas HMT_930,1.3118687
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae,Alloprevotella HMT_473,1.2402106
k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Corynebacteriales.f__Corynebacteriaceae,Corynebacterium amycolatum,1.1755583
k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae,Prevotella oris,0.8161724
k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Propionibacteriales.f__Propionibacteriaceae,Arachnia rubra,0.7956432


Name,Genus_Species,MeanDecreaseGini
<chr>,<chr>,<dbl>
d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Staphylococcales.f__Gemellaceae,Gemella,1.3922065
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae,Muricomes_149725,1.0053027
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__UBA1381.f__UBA1381,CAG.41 CAG.41.sp001941225,0.9189178
Unassigned.__.__.__.__.__.__,NA,0.7634532
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Oscillospirales.f__Acutalibacteraceae,UMGS1071 UMGS1071.sp900548305,0.6288163
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae,Blautia_A_141780 Blautia_A_141780.argi,0.5280272


Name,Genus_Species,MeanDecreaseGini
<chr>,<chr>,<dbl>
d__Bacteria.p__Firmicutes_D.c__Bacilli.o__Staphylococcales.f__Gemellaceae,Gemella,1.1441064
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__UBA1381.f__UBA1381,CAG.41 CAG.41.sp001941225,0.8719698
Unassigned.__.__.__.__.__.__,NA,0.8167096
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Eubacteriales_258471.f__Eubacteriaceae_258449,Eubacterium_O_258270,0.6964246
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae,Muricomes_149725,0.6398613
d__Bacteria.p__Firmicutes_A.c__Clostridia_258483.o__Lachnospirales.f__Lachnospiraceae,Blautia_A_141780 Blautia_A_141780.argi,0.5979297


# Support Vector Machine (SVM)
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predicts by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n
- Compared to other classification algorithms, this approach can reliably classify chemicals while avoiding overfitting and reducing susceptibility to noisy or meaningless data

In [12]:
# the caret fn prefers for the outcome variable to have letters instead of numbers so reformatting the df
v1_saliva_df = v1_saliva_df %>%
     mutate(Outcome = relevel(factor(ifelse(Outcome %in% '1', 'SU', 'F')), ref = "F"))
v1_saliva_cov_df = v1_saliva_cov_df %>%
     mutate(Outcome = relevel(factor(ifelse(Outcome %in% '1', 'SU', 'F')), ref = "F"))
v1_stool_df = v1_stool_df %>%
     mutate(Outcome = relevel(factor(ifelse(Outcome %in% '1', 'SU', 'F')), ref = "F"))
v1_stool_cov_df = v1_stool_cov_df %>%
     mutate(Outcome = relevel(factor(ifelse(Outcome %in% '1', 'SU', 'F')), ref = "F"))

v2_saliva_df = v2_saliva_df %>%
     mutate(IgE_Binarized = relevel(factor(ifelse(IgE_Binarized %in% '1', 'High', 'Low')), ref = "Low"))
v2_saliva_cov_df = v2_saliva_cov_df %>%
     mutate(IgE_Binarized = relevel(factor(ifelse(IgE_Binarized %in% '1', 'High', 'Low')), ref = "Low"))
v2_stool_df = v2_stool_df %>%
     mutate(IgE_Binarized = relevel(factor(ifelse(IgE_Binarized %in% '1', 'High', 'Low')), ref = "Low"))
v2_stool_cov_df = v2_stool_cov_df %>%
     mutate(IgE_Binarized = relevel(factor(ifelse(IgE_Binarized %in% '1', 'High', 'Low')), ref = "Low"))

In [13]:
# choosing parameters to be tuned for linear svm only
tunegrid_linear_svm = data.frame(C = seq(from = 0.01, to = 0.1, by = 0.01))

svm_classification = function(dataset, model, outcome, pred_outcome, tunegrid, input_sample, covariates, knn){

    #setting seed for reproducibility
    set.seed(12)
    
    # 5 fold CV first
    control = trainControl(method = 'cv', number = 5, search = 'grid', classProbs = TRUE, savePredictions = TRUE,
        summaryFunction = twoClassSummary)

    # Establish a list of indices that will used to identify our training and testing data with a 60-40 split
    dataset_index = createDataPartition(y = dataset[[outcome]], p = 0.6, list = FALSE)
    
    # Use indices to make our training and testing datasets and view the number of Ds and NDs
    data_train = dataset[dataset_index,]
    
    # using SMOTE to balance the class distribution
    balanced_data_train = smotenc(data_train, outcome, k = knn)
    data_test = dataset[-dataset_index,]

    # now pruning parameters (based on the training dataset to prevent overfitting)
    svm_tune = train(form = Outcome~., data = balanced_data_train, method = model, trControl = control,
         metric = "ROC", # this gives me the same as balanced accuracy but it isn't able to find that ?
         maximize = TRUE, tuneGrid = tunegrid) 
    
    # predicting with tune parameters
    data_test[[pred_outcome]] = predict(svm_tune, newdata = data_test)
    
    # always specify the correct positive class when calculating the confusion matrix
    matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]],
                                 positive = "SU")

    #calculating AUC
    AUC = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
    
    #extracting accuracy, sens, spec, PPV, NPV to take mean later
    metrics = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:4])), AUC) %>%
        rename(PPV = Pos.Pred.Value, NPV = Neg.Pred.Value) %>%
        mutate(Model = "SVM", Response = outcome, Sample = input_sample, Covariates = covariates, Kernel = model)  

    metrics = metrics[,c(7:11,1:6)]

    # getting variable importance
    svm_best = svm_tune$finalModel

    coefs = svm_best@coef[[1]]
    mat = svm_best@xmatrix[[1]]
    
    variable_importance_df = data.frame(t(coefs %*% mat)) %>%
        rename(`Variable Importance` = t.coefs.....mat.) %>%
        arrange(-`Variable Importance`) %>%
        mutate(Sample = input_sample, Covariates = covariates, Kernel = model, Passed_Filter = NA) %>%
        rownames_to_column(var = "Predictor")

    return(list(metrics, variable_importance_df))

}

In [14]:
#calling fn
svm_saliva_bl_linear = svm_classification(v1_saliva_df, "svmLinear", "Outcome", "pred_Outcome",
                                            tunegrid_linear_svm, 'Saliva', 'No', 5)
svm_saliva_bl_radial = svm_classification(v1_saliva_df, "svmRadial", "Outcome", "pred_Outcome",
                                            NULL, 'Saliva', 'No', 5)
svm_saliva_bl_polynomial = svm_classification(v1_saliva_df, "svmPoly", "Outcome", "pred_Outcome",
                                            NULL, 'Saliva', 'No', 5)
svm_stool_bl_linear = svm_classification(v1_stool_df, "svmLinear", "Outcome", "pred_Outcome",
                                            tunegrid_linear_svm, 'Stool', 'No', 3)
svm_stool_bl_radial = svm_classification(v1_stool_df, "svmRadial", "Outcome", "pred_Outcome",
                                            NULL, 'Stool', 'No', 3)
svm_stool_bl_polynomial = svm_classification(v1_stool_df, "svmPoly", "Outcome", "pred_Outcome",
                                            NULL, 'Stool', 'No', 3)
svm_saliva_cov_linear = svm_classification(v1_saliva_cov_df, "svmLinear", "Outcome", "pred_Outcome",
                                            tunegrid_linear_svm, 'Saliva', 'Yes', 5)
svm_saliva_cov_radial = svm_classification(v1_saliva_cov_df, "svmRadial", "Outcome", "pred_Outcome",
                                            NULL, 'Saliva', 'Yes', 5)
svm_saliva_cov_polynomial = svm_classification(v1_saliva_cov_df, "svmPoly", "Outcome", "pred_Outcome",
                                            NULL, 'Saliva', 'Yes', 5)
svm_stool_cov_linear = svm_classification(v1_stool_cov_df, "svmLinear", "Outcome", "pred_Outcome",
                                            tunegrid_linear_svm, 'Stool', 'Yes', 3)
svm_stool_cov_radial = svm_classification(v1_stool_cov_df, "svmRadial", "Outcome", "pred_Outcome",
                                            NULL, 'Stool', 'Yes', 3)
svm_stool_cov_polynomial = svm_classification(v1_stool_cov_df, "svmPoly", "Outcome", "pred_Outcome",
                                            NULL, 'Stool', 'Yes', 3)

Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x,

In [15]:
# choosing parameters to be tuned for linear svm only
tunegrid_linear_svm = data.frame(C = seq(from = 0.01, to = 0.1, by = 0.01))

svm_classification = function(dataset, model, outcome, pred_outcome, tunegrid, input_sample, covariates, knn){

    #setting seed for reproducibility
    set.seed(12)
    
    # 5 fold CV first
    control = trainControl(method = 'cv', number = 5, search = 'grid', classProbs = TRUE, savePredictions = TRUE,
        summaryFunction = twoClassSummary)

    # Establish a list of indices that will used to identify our training and testing data with a 60-40 split
    dataset_index = createDataPartition(y = dataset[[outcome]], p = 0.6, list = FALSE)
    
    # Use indices to make our training and testing datasets and view the number of Ds and NDs
    data_train = dataset[dataset_index,]
    
    # using SMOTE to balance the class distribution
    balanced_data_train = smotenc(data_train, outcome, k = knn)
    data_test = dataset[-dataset_index,]

    # now pruning parameters (based on the training dataset to prevent overfitting)
    svm_tune = train(form = IgE_Binarized~., data = balanced_data_train, method = model, trControl = control,
         metric = "ROC", # this gives me the same as balanced accuracy but it isn't able to find that ?
         maximize = TRUE, tuneGrid = tunegrid) 
    
    # predicting with tune parameters
    data_test[[pred_outcome]] = predict(svm_tune, newdata = data_test)
    
    # always specify the correct positive class when calculating the confusion matrix
    matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]],
                                 positive = "High")

    #calculating AUC
    AUC = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
    
    #extracting accuracy, sens, spec, PPV, NPV to take mean later
    metrics = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:4])), AUC) %>%
        rename(PPV = Pos.Pred.Value, NPV = Neg.Pred.Value) %>%
        mutate(Model = "SVM", Response = outcome, Sample = input_sample, Covariates = covariates, Kernel = model)   

    metrics = metrics[,c(7:11,1:6)]

    # getting variable importance
    svm_best = svm_tune$finalModel

    coefs = svm_best@coef[[1]]
    mat = svm_best@xmatrix[[1]]
    
    variable_importance_df = data.frame(t(coefs %*% mat)) %>%
        rename(`Variable Importance` = t.coefs.....mat.) %>%
        arrange(-`Variable Importance`) %>%
        mutate(Sample = input_sample, Covariates = covariates, Kernel = model, Passed_Filter = NA) %>%
        rownames_to_column(var = "Predictor")

    return(list(metrics, variable_importance_df))

}

In [16]:
svm_saliva_ige_linear = svm_classification(v2_saliva_df, "svmLinear", "IgE_Binarized", "pred_IgE",
                                            tunegrid_linear_svm, 'Saliva', 'No', 5)
svm_saliva_ige_radial = svm_classification(v2_saliva_df, "svmRadial", "IgE_Binarized", "pred_IgE",
                                            NULL, 'Saliva', 'No', 5)
svm_saliva_ige_polynomial = svm_classification(v2_saliva_df, "svmPoly", "IgE_Binarized", "pred_IgE",
                                            NULL, 'Saliva', 'No', 5)
svm_stool_ige_linear = svm_classification(v2_stool_df, "svmLinear", "IgE_Binarized", "pred_IgE",
                                            tunegrid_linear_svm, 'Stool', 'No', 5)
svm_stool_ige_radial = svm_classification(v2_stool_df, "svmRadial", "IgE_Binarized", "pred_IgE",
                                            NULL, 'Stool', 'No', 5)
svm_stool_ige_polynomial = svm_classification(v2_stool_df, "svmPoly", "IgE_Binarized", "pred_IgE",
                                            NULL, 'Stool', 'No', 5)
svm_saliva_ige_cov_linear = svm_classification(v2_saliva_cov_df, "svmLinear", "IgE_Binarized", "pred_IgE",
                                            tunegrid_linear_svm, 'Saliva', 'Yes', 5)
svm_saliva_ige_cov_radial = svm_classification(v2_saliva_cov_df, "svmRadial", "IgE_Binarized", "pred_IgE",
                                            NULL, 'Saliva', 'Yes', 5)
svm_saliva_ige_cov_polynomial = svm_classification(v2_saliva_cov_df, "svmPoly", "IgE_Binarized", "pred_IgE",
                                            NULL, 'Saliva', 'Yes', 5)
svm_stool_ige_cov_linear = svm_classification(v2_stool_cov_df, "svmLinear", "IgE_Binarized", "pred_IgE",
                                            tunegrid_linear_svm, 'Stool', 'Yes', 5)
svm_stool_ige_cov_radial = svm_classification(v2_stool_cov_df, "svmRadial", "IgE_Binarized", "pred_IgE",
                                            NULL, 'Stool', 'Yes', 5)
svm_stool_ige_cov_polynomial = svm_classification(v2_stool_cov_df, "svmPoly", "IgE_Binarized", "pred_IgE",
                                            NULL, 'Stool', 'Yes', 5)

Setting levels: control = Low, case = High

Warning message in value[[3L]](cond):
“Ordered predictor converted to numeric vector. Threshold values will not correspond to values in predictor.”
Setting direction: controls < cases

Setting levels: control = Low, case = High

Warning message in value[[3L]](cond):
“Ordered predictor converted to numeric vector. Threshold values will not correspond to values in predictor.”
Setting direction: controls < cases

Setting levels: control = Low, case = High

Warning message in value[[3L]](cond):
“Ordered predictor converted to numeric vector. Threshold values will not correspond to values in predictor.”
Setting direction: controls < cases

Setting levels: control = Low, case = High

Warning message in value[[3L]](cond):
“Ordered predictor converted to numeric vector. Threshold values will not correspond to values in predictor.”
Setting direction: controls < cases

Setting levels: control = Low, case = High

Warning message in value[[3L]](cond):
“O

In [19]:
final_df = rbind(rf_confusion_matrix_bl, svm_saliva_bl_linear[[1]], svm_saliva_bl_radial[[1]], svm_saliva_bl_polynomial[[1]],
                 svm_saliva_cov_linear[[1]], svm_saliva_cov_radial[[1]], svm_saliva_bl_polynomial[[1]],
                rf_confusion_matrix_ige, svm_saliva_ige_linear[[1]], svm_saliva_ige_radial[[1]], svm_saliva_ige_polynomial[[1]],
                svm_saliva_ige_cov_linear[[1]], svm_saliva_ige_cov_radial[[1]], svm_saliva_ige_cov_polynomial[[1]])

final_df

Model,Response,Sample,Covariates,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,NPV,AUC
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,Outcome,Saliva,No,NA,0.6500000,0.5000000,0.8000000,0.7333333,0.6333333,0.7500000
RF Classification,Outcome,Saliva,Yes,NA,0.5500000,0.5000000,0.6000000,0.6333333,0.5333333,0.7500000
RF Classification,Outcome,Stool,No,NA,0.5000000,0.6000000,0.4000000,0.6333333,NaN,0.7000000
RF Classification,Outcome,Stool,Yes,NA,0.5500000,0.6000000,0.5000000,0.6666667,NaN,0.7500000
SVM,Outcome,Saliva,No,svmLinear,0.4166667,0.5000000,0.3333333,0.5000000,0.3333333,0.5833333
SVM,Outcome,Saliva,No,svmRadial,0.5833333,0.5000000,0.6666667,0.6666667,0.5000000,0.5833333
SVM,Outcome,Saliva,No,svmPoly,0.5833333,0.5000000,0.6666667,0.6666667,0.5000000,0.5833333
SVM,Outcome,Saliva,Yes,svmLinear,0.2916667,0.2500000,0.3333333,0.3333333,0.2500000,0.7083333
SVM,Outcome,Saliva,Yes,svmRadial,0.2916667,0.2500000,0.3333333,0.3333333,0.2500000,0.7083333


## Comparing RF Importance and Individual Significance

Seeing if bacteria that were significant individually are still showing a high level of significance at the global level.

In [10]:
saliva_outcome_df = data.frame(read_excel("Input/Saliva_Outcome_112025.xlsx")) %>%
    filter(`P.Adj` < 0.1) 
stool_outcome_df = data.frame(read_excel("Input/Stool_Outcome_112025.xlsx")) %>%
    filter(`P.Adj` < 0.1) 
saliva_adj_outcome_df = data.frame(read_excel("Input/Saliva_Adj_Outcome_112025.xlsx")) %>%
    filter(`P.Adj` < 0.1)
stool_adj_outcome_df = data.frame(read_excel("Input/Stool_Adj_Outcome_112025.xlsx")) %>%
    filter(`P.Adj` < 0.1) #%>%
    #unite("Name", Name, Name1, sep = '.g__')

saliva_ige_df = data.frame(read_excel("Input/Saliva_IgE_112025.xlsx")) %>%
    filter(`P.Adj` < 0.1) 
stool_ige_df = data.frame(read_excel("Input/Stool_IgE_112025.xlsx")) %>%
    filter(`P.Adj` < 0.1) 
saliva_adj_ige_df = data.frame(read_excel("Input/Saliva_Adj_IgE_112025.xlsx")) %>%
    filter(`P.Adj` < 0.1) 
stool_adj_ige_df = data.frame(read_excel("Input/Stool_Adj_IgE_112025.xlsx")) %>%
    filter(`P.Adj` < 0.1)# %>%
    #unite("Name", Name, Name1, sep = '-')

head(saliva_outcome_df)

,Name,logFC,P.Value,P.Adj,Sensitivity.Analysis,diff_robust_OutcomeSU
,<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>
1,k__Bacteria.p__Actinobacteria.c__Actinomycetia.o__Actinomycetales.f__Actinomycetaceae.g__Actinomyces.s__graevenitzii,-1.866753,0.0028450021,0.08321631,TRUE,FALSE
2,k__Bacteria.p__Bacteroidetes.c__Bacteroidia.o__Bacteroidales.f__Prevotellaceae.g__Alloprevotella.s__sp._HMT_308,-2.955958,0.0006104559,0.04276485,FALSE,FALSE
3,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Leptotrichia.__,-1.853613,0.0007310231,0.04276485,FALSE,FALSE
4,k__Bacteria.p__Fusobacteria.c__Fusobacteriia.o__Fusobacteriales.f__Leptotrichiaceae.g__Pseudoleptotrichia.s__sp._HMT_221,2.616246,0.0025283095,0.08321631,TRUE,FALSE


In [19]:
bl_saliva_sig_df = rf_values_bl_saliva[[3]] %>%
    mutate(Individual_Sig = ifelse(Predictor %in% saliva_outcome_df$Name, 'Yes', 'No'))
bl_stool_sig_df = rf_values_bl_stool[[3]] %>%
    mutate(Individual_Sig = ifelse(Predictor %in% stool_outcome_df$Name, 'Yes', 'No')) 
bl_saliva_cov_sig_df = rf_values_bl_saliva_covariates[[3]] %>%
    mutate(Individual_Sig = ifelse(Predictor %in% saliva_adj_outcome_df$Name, 'Yes', 'No')) 
bl_stool_cov_sig_df = rf_values_bl_stool_covariates[[3]] %>%
    mutate(Individual_Sig = ifelse(Predictor %in% stool_adj_outcome_df$Name, 'Yes', 'No')) 

saliva_ige_sig_df = rf_values_ige_saliva[[3]] %>%
    mutate(Individual_Sig = ifelse(Predictor %in% saliva_ige_df$Name, 'Yes', 'No')) 
stool_ige_sig_df = rf_values_ige_stool[[3]] %>%
    mutate(Individual_Sig = ifelse(Predictor %in% stool_ige_df$Name, 'Yes', 'No')) 
saliva_ige_cov_sig_df = rf_values_ige_saliva_covariates[[3]] %>%
    mutate(Individual_Sig = ifelse(Predictor %in% saliva_adj_ige_df$Name, 'Yes', 'No')) 
stool_ige_cov_sig_df = rf_values_ige_stool_covariates[[3]] %>%
    mutate(Individual_Sig = ifelse(Predictor %in% stool_adj_ige_df$Name, 'Yes', 'No')) 

In [20]:
# exporting
write.xlsx(bl_saliva_sig_df, paste0(Output,"/", "RF_Saliva_Outcome_Importance_", cur_date, ".xlsx"), 
            rowNames = FALSE)
write.xlsx(bl_stool_sig_df, paste0(Output,"/", "RF_Stool_Outcome_Importance_", cur_date, ".xlsx"), 
            rowNames = FALSE)
write.xlsx(bl_saliva_cov_sig_df, paste0(Output,"/", "RF_Saliva_Covariates_Outcome_Importance_", cur_date, ".xlsx"), 
            rowNames = FALSE)
write.xlsx(bl_stool_cov_sig_df, paste0(Output,"/", "RF_Stool_Covariates_Outcome_Importance_", cur_date, ".xlsx"), 
           rowNames = FALSE)

write.xlsx(saliva_ige_sig_df, paste0(Output,"/", "RF_Saliva_IgE_Importance_", cur_date, ".xlsx"), 
            rowNames = FALSE)
write.xlsx(stool_ige_sig_df, paste0(Output,"/", "RF_Stool_IgE_Importance_", cur_date, ".xlsx"), 
            rowNames = FALSE)
write.xlsx(saliva_ige_cov_sig_df, paste0(Output,"/", "RF_Saliva_Covariates_IgE_Importance_", cur_date, ".xlsx"), 
            rowNames = FALSE)
write.xlsx(stool_ige_cov_sig_df, paste0(Output,"/", "RF_Stool_Covariates_IgE_Importance_", cur_date, ".xlsx"), 
           rowNames = FALSE)